![example](images/director_shot.jpeg)

# Current Movie Trends Analysis for Microsoft Studios

**Authors:** Jonathan, Matt, Nate, Roshni
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

"This project analyzes multiple movie databases containing datapoints across thousands of movies. Descriptive analysis of budget, runtime, genre, and rating data shows that certain types of movies typically outperform others. Microsoft's new movie studio can use this analysis to guide movie production decisions. We found **x info about budget**, movies that have a rating of **x** and above typically have a runtime in the 85-110 min range, and documentaries to be the genre with the highest average review."

* **Which methods did we use?**

* **What are the recommendations?**

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

"Microsoft may be able to make an educated decision for deciding the criteria of their first film by using data from other movie releases. Doing so will give their movie the greatest likelihood of being a box office success.

Using data from IMDB, TheMovieDB, and The Numbers, we describe patterns across budget decisions, movie runtime and genre decisions, and movie ratings. We chose this data to highlight expected budget costs and movie genres for movies that have been voted as the highest rated by viewers."

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Data for this analysis is taken from 3 of the largest online movie databases with datapoints on hundreds of thousands of movies. Below, we further break down the sources and relevant information from each.

***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### 1: The MovieDB (TMDB)

TMDB dataset comprises of records on 26,517 movies and includes 9 columns of datapoints on each movie. The target data includes release date and genre where the genre codes are ordered by relevance.

In [2]:
df_tmdb = pd.read_csv('./zippedData/tmdb.movies.csv.gz', index_col = 0, parse_dates=['release_date'])

In [3]:
df_tmdb.head(3)

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368


### 2: The Numbers (TN)

TN dataset comprises records on **Fill out the rest later**

In [4]:
tn_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz', parse_dates=['release_date'])

In [5]:
tn_df.head(3)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"


### 3: IMDB

IMDB dataset comprises of multiple tables containing various information about movies. The two tables of interest are: `movie_basics` and `movie_ratings`.

The `movie_basics` table includes movie titles, release year, and genres. The `movie_ratings` table includes average movie rating and number of votes. The primary key for both tables is `movie_id`.

Target data include runtime minutes and average rating.

**NOTE:** In order to run this notebook, you will need to download the necessary data file. A zipped version of the file can be downloaded: **here**.

1) Download the zipped file into a folder called 'zippedData' within the same folder that this notebook is located. Once this zip file is in located in the zippedData folder, you can run the following cell to unzip it.

In [6]:
import zipfile
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('zippedData')

2) Run the following cell to confirm that the database file has been downloaded and unzipped to the correct folder:

In [7]:
from os.path import exists
file_exists = exists('zippedData/im.db')
if file_exists:
    print("You have successfully downloaded and unzipped the database file.  Please proceed.")
else:
    print("The database file was not found.  Please ensure that the database file has been downloaded and unzipped into the 'zippedData' folder located in the folder that this Jupyter notebook resides.")

You have successfully downloaded and unzipped the database file.  Please proceed.


3) Next, we want to open up the database and examine the database tables.

In [8]:
import sqlite3

con = sqlite3.connect('zippedData/im.db')
query = """  SELECT * FROM sqlite_master  """
tables = pd.read_sql(query, con)
tables

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


#### Viewing the tables of interest:

Movie basics Table:

In [9]:
query = """  SELECT * FROM movie_basics  """
movie_basics = pd.read_sql(query, con)
movie_basics.head(3)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama


Movie Ratings Table:

In [10]:
query = """  SELECT * FROM movie_ratings  """
movie_ratings = pd.read_sql(query, con)
movie_ratings.head(3)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20


## Data Preparation

Using three sources of data for this analysis required multiple strategies for cleaning our data. Below, we discuss our process and reasoning for each set.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

### 1. The Movies Database
#### 1.1 Removing Unnecessary Columns & Filtering Data
**Columns to drop:**
- `popularity`
   - Popularity is based on current website popularity, not how well the movie performed when released or how it was reviewed
        - https://developers.themoviedb.org/3/getting-started/popularity
    
- `id`
    - Just a unique identifier, not relevant

- `original_title`
    - There are two columns for title, we will keep the final title column

The `df_cleaning` variable is created to store the dataframe after dropping the columns.

In [11]:
df_cleaning = df_tmdb.drop(['popularity', 'id', 'original_title'], axis=1)

**Columns to filter:**
- `original_language`
    - Microsoft would make the language in english as the company is based out of the USA
    
- `vote_Count`
    - Microsoft is only interested in a successful movie
    - It can be reasonably assumed that movies with fewer votes are less successful
    - We are dropping any count values below the mean
    
- `release_date`
    - Microsoft is interested in current data trends
        - We will limit the data to the last 10 years of available data (2010-2019)

The `df_filtered` variable is created to store the cleaned and filtered dataframe.

**Removing non-English language films.**

In [12]:
df_filtered = df_cleaning[df_cleaning['original_language'] == 'en']
df_filtered = df_filtered.drop('original_language', axis=1)

**Drop Certain Movies**

Dropping movies with a vote count below 200 votes.
(We chose this value as it is roughly the average of the number of votes in the dataset)

In [13]:
df_filtered['vote_count'].describe()

count    23291.000000
mean       209.307887
std       1016.214512
min          1.000000
25%          1.000000
50%          4.000000
75%         24.000000
max      22186.000000
Name: vote_count, dtype: float64

In [14]:
df_filtered = df_filtered[df_filtered['vote_count'] > 200]

Limiting the number of movies to those released in 2010 and after.

In [15]:
df_filtered = df_filtered[df_filtered['release_date'] > pd.Timestamp(2010, 1, 1)]

We are left with the table `df_filtered` that has 2318 rows and 5 columns of relevant information

#### 1.2 Converting the Data

**Making sense of the genre_ids**

We can see that the genre ID's are in order of best fit rather than numerical or alphabetical order
- For index 0: `genre_ids` = [12, 14, 10751]
- For index 1: `genre_ids` = [14, 12, 16, 10751]

We will take the primary `genre_ids` from the list to isolate the most relevant genre.

**Determining the data type for genre_ids**

Checking the type of data for genre_ids below. Goal is to create new columns with primary and secondary genres.

In [16]:
print(df_filtered['genre_ids'][0])
print(type(df_filtered['genre_ids'][0]))

[12, 14, 10751]
<class 'str'>


**Created a for-loop that runs through each row of the column** `genre_ids`

The data in `genre_ids` looks like a list, but is really a string including brackets. We are able to clean this with a for-loop.

The for-loop:
 - First converts the string to a list of int.
 - Then it takes each int and assigns it to a new list corresponding to it's position in the original `genre_ids` list.
 - If there is no value for that iteration, it populates the list with `None`.
 - I include nonetype so that they stay on point with the correct index.

In [17]:
char_remove = ["'", " ", "[", "]"]

first_genre = []

for x in df_filtered['genre_ids']:
    row = x
    for char in char_remove:
        row = row.replace(char, '')
    row = row.split(',')
    first_genre.append(row[0])

#### 1.3 Adding New Columns

**Now that we have lists of primary genre's we can turn them into columns.**

In [18]:
df_filtered['primary_genre'] = first_genre
df_filtered.head()

,genre_ids,release_date,title,vote_average,vote_count,primary_genre
0,"[12, 14, 10751]",2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,12
1,"[14, 12, 16, 10751]",2010-03-26,How to Train Your Dragon,7.7,7610,14
2,"[12, 28, 878]",2010-05-07,Iron Man 2,6.8,12368,12
4,"[28, 878, 12]",2010-07-16,Inception,8.3,22186,28
5,"[12, 14, 10751]",2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229,12


**Converting the genre_ids codes to something more understandable**

We found the below movie genre key on The MovieDB website. The key allows us to convert the `genre_ids` coding into standard English.

(found at: https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee)

In [19]:
backwards_key = {
'Action' : '28',
'Adventure' : '12',
'Animation' : '16',
'Comedy' : '35',
'Crime' : '80',
'Documentary' : '99',
'Drama' : '18',
'Family' : '10751',
'Fantasy' : '14',
'History' : '36',
'Horror' : '27',
'Music' : '10402',
'Mystery' : '9648',
'Romance' : '10749',
'Science Fiction' : '878',
'TV Movie' : '10770',
'Thriller' : '53',
'War' : '10752',
'Western' : '37'
}

genre_key = {v: k for k, v in backwards_key.items()}
# print(genre_key)

**Now to put the key to work and change the `primary_genre` column from numbers to English.**

In [20]:
df_filtered["primary_genre"].replace(genre_key, inplace=True)
df_filtered.head()

,genre_ids,release_date,title,vote_average,vote_count,primary_genre
0,"[12, 14, 10751]",2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,Adventure
1,"[14, 12, 16, 10751]",2010-03-26,How to Train Your Dragon,7.7,7610,Fantasy
2,"[12, 28, 878]",2010-05-07,Iron Man 2,6.8,12368,Adventure
4,"[28, 878, 12]",2010-07-16,Inception,8.3,22186,Action
5,"[12, 14, 10751]",2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229,Adventure


**For the last step before analysis, we'll change the type of `primary_genre` to categorical information.**

In [21]:
df_filtered['primary_genre'] = df_filtered['primary_genre'].astype('category')

### 2. Data Prep The Numbers Database

#### 2.1 Removing Unnecessary Columns & Filtering Data
**Columns to drop:**
- `id`
    - Just a unique identifier, not relevant

In [22]:
tn_df = tn_df.drop('id', axis=1)

**Columns to filter:**
- `worldwide_gross`
    - As a large company looking for a box office hit, we want only movies that have a worldwide gross value

#### Convert Strings of numbers in each column to Int type
First define a function and then clean each column

In [23]:
def clean_nums(column_title):
    '''
    Input a column title as a string to remove commas and dollar signs. Return the column as type int.
    '''
    tn_df[column_title].replace(',','', regex=True, inplace=True)
    tn_df[column_title] = tn_df[column_title].map(lambda x: int(x.replace('$','')))
    
    return tn_df

In [24]:
clean_nums('production_budget')
clean_nums('domestic_gross')
clean_nums('worldwide_gross')

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,2009-12-18,Avatar,425000000,760507625,2776345279
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...
5777,2018-12-31,Red 11,7000,0,0
5778,1999-04-02,Following,6000,48482,240495
5779,2005-07-13,Return to the Land of Wonders,5000,1338,1338
5780,2015-09-29,A Plague So Pleasant,1400,0,0


In [25]:
# tn_df['production_budget'].replace(',','', regex=True, inplace=True)
# tn_df['production_budget'] = tn_df['production_budget'].map(lambda x: x.replace('$',''))

# tn_df['domestic_gross'].replace(',','', regex=True, inplace=True)
# tn_df['domestic_gross'] = tn_df['domestic_gross'].map(lambda x: x.replace('$',''))

# tn_df['worldwide_gross'].replace(',','', regex=True, inplace=True)
# tn_df['worldwide_gross'] = tn_df['worldwide_gross'].map(lambda x: x.replace('$',''))

# tn_df['production_budget'] = tn_df['production_budget'].astype(int)
# tn_df['domestic_gross'] = tn_df['domestic_gross'].astype(int)
# tn_df['worldwide_gross'] = tn_df['worldwide_gross'].astype(int)

# tn_df

#### Excluding Worldwide Gross == 0
- Worldwide Gross of 0 indicates that either the data is missing data or the movie was never released in theaters.

In [22]:
print("Excluding worldwide gross of $0 would remove " + str(round((1 - 5415/5781)*100, 1)) + " percent of datapoints")

#Less than 10% of data is being removed, which meets the rule of thumb


tn_df = tn_df[tn_df['worldwide_gross'] != 0]

tn_df

Excluding worldwide gross of $0 would remove 6.3 percent of datapoints


,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,2009-12-18,Avatar,425000000,760507625,2776345279
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...
5775,2006-05-26,Cavite,7000,70071,71644
5776,2004-12-31,The Mongol King,7000,900,900
5778,1999-04-02,Following,6000,48482,240495
5779,2005-07-13,Return to the Land of Wonders,5000,1338,1338


#### Calculate Net Profit as a new variable in dataset

Net profit = worldwide gross - production budget

In [23]:
tn_df = tn_df.assign(net_profit = tn_df['worldwide_gross'] - tn_df['production_budget'])
tn_df

,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_profit
0,2009-12-18,Avatar,425000000,760507625,2776345279,2351345279
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875
2,2019-06-07,Dark Phoenix,350000000,42762350,149762350,-200237650
3,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963
4,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747
...,...,...,...,...,...,...
5775,2006-05-26,Cavite,7000,70071,71644,64644
5776,2004-12-31,The Mongol King,7000,900,900,-6100
5778,1999-04-02,Following,6000,48482,240495,234495
5779,2005-07-13,Return to the Land of Wonders,5000,1338,1338,-3662


### Data Prep The IMDb

Opening up the database and examining the tables:

In order to run this notebook, you will need to download the necessary data file. A zipped version of the file can be downloaded: here.

Download the zipped file into a folder called 'zippedData' within the same folder that this notebook is located. Once this zip file is in located in the zippedData folder, you can run the following cell to unzip it.

In [35]:
import zipfile
with zipfile.ZipFile('zippedData/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('zippedData')

Run the following cell to confirm that the database file has been downloaded and unzipped to the correct folder:

In [37]:
from os.path import exists
file_exists = exists('zippedData/im.db')
if file_exists:
    print("You have successfully downloaded and unzipped the database file.  Please proceed.")
else:
    print("The database file was not found.  Please ensure that the database file has been downloaded and unzipped into the 'zippedData' folder located in the folder that this Jupyter notebook resides.")

You have successfully downloaded and unzipped the database file.  Please proceed.


First, we want to open up the database and examine the database tables.

In [33]:
import sqlite3

con = sqlite3.connect('zippedData/im.db')
query = """  SELECT * FROM sqlite_master  """
tables = pd.read_sql(query, con)
tables

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


**Tables of initial interest:**
1. movie_basics
2. movie_ratings

#### Examine Movie Basics table

In [26]:
query = """  SELECT * FROM movie_basics  """
movie_basics = pd.read_sql(query, con)
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [27]:
movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


**Notes:**
- This table has the movie titles, release year, and genres.  The primary key is the movie_id, which is referenced in other tables such as movie basics.
- There appears to be a movie_id, primary title, and start_year for all rows.

#### Examine Movie Ratings table

In [28]:
query = """  SELECT * FROM movie_ratings  """
movie_ratings = pd.read_sql(query, con)
movie_ratings

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [29]:
movie_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


**Notes:**
- Movie ratings are only given for 73K of the 146K of movies.  
- Since the ratings are the most important quantitate information from this file, I would suggest we 

#### Merge Tables
- merging `movie_ratings` with `movie_basics` 

In [30]:
movies_with_ratings = pd.merge(movie_ratings, movie_basics, left_on='movie_id', right_on='movie_id')
movies_with_ratings

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,Borderless,Borderless,2019,87.0,Documentary
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"
...,...,...,...,...,...,...,...,...
73851,tt9805820,8.1,25,Caisa,Caisa,2018,84.0,Documentary
73852,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120.0,"Action,Animation,Sci-Fi"
73853,tt9851050,4.7,14,Sisters,Sisters,2019,NaN,"Action,Drama"
73854,tt9886934,7.0,5,The Projectionist,The Projectionist,2019,81.0,Documentary


#### Filter Movies
We want to consider the number of votes in our results, as the fewer number of votes a movie rating has will lead to less certainty in the average rating. As such, we decided to cut the analysis to the top half (all movies over the median number of votes, which is 49).

Find the median number of votes in the remaining data set

In [41]:
median_votes = movies_with_ratings['numvotes'].median()
median_votes

49.0

Drop movies with number of votes less than the median

In [44]:
movies_with_ratings_over_median_votes = movies_with_ratings.loc[movies_with_ratings['numvotes']>median_votes]
movies_with_ratings_over_median_votes

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres
1,tt10384606,8.9,559,Borderless,Borderless,2019,87.0,Documentary
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
5,tt1069246,6.2,326,Habana Eva,Habana Eva,2010,106.0,"Comedy,Romance"
6,tt1094666,7.0,1613,The Hammer,Hamill,2010,108.0,"Biography,Drama,Sport"
7,tt1130982,6.4,571,The Night Clerk,Avant l'aube,2011,104.0,"Drama,Thriller"
...,...,...,...,...,...,...,...,...
73831,tt9558612,3.7,4057,PM Narendra Modi,PM Narendra Modi,2019,136.0,"Biography,Drama"
73832,tt9575726,7.7,161,Ir visi ju vyrai,Ir visi ju vyrai,2019,108.0,"Comedy,Romance"
73833,tt9576110,6.8,65,J'veux du soleil,J'veux du soleil,2019,76.0,Documentary
73844,tt9668598,5.5,317,Musabbar,Musabbar,2019,NaN,"Horror,Thriller"


We now have a dataset of ~37K movies that all have a sufficient number of votes for us to be confident in the average ratings. We can now analyze the dataset.

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [32]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***